# Autogen code executors

From the documentation about code executors:

"In AutoGen, a code executor is a component that takes input messages (e.g., those containing code blocks), performs execution, and outputs messages with the results. AutoGen provides two types of built-in code executors, one is command line code executor, which runs code in a command line environment such as a UNIX shell, and the other is Jupyter executor, which runs code in an interactive Jupyter kernel."

References:

- [Code executors](https://microsoft.github.io/autogen/docs/tutorial/code-executors/)

## Install the required packages

In [ ]:
%pip install -q python-dotenv==1.0.1 openai==1.35.9 gradio==4.39.0 pyautogen==0.2.32

## Import the required packages and create the Azure OpenAI client

In [2]:
import os
import pprint
from pathlib import Path
from autogen import ConversableAgent
from autogen.coding import CodeBlock, LocalCommandLineCodeExecutor
from settings import Settings
settings = Settings()

## Test the local code executor

- The executore is capable of executing code and returning the results

In [ ]:
work_dir = Path("coding")
work_dir.mkdir(exist_ok=True)

executor = LocalCommandLineCodeExecutor(work_dir=work_dir)
print(
    executor.execute_code_blocks(
        code_blocks=[
            CodeBlock(language="python", code="print('Hello Word!')\nfor i in range(0,10,2):\n  print(i)"),
        ]
    )
)

## Config an Autogen conversable agent

In [4]:
config_list = [
    {
        "base_url": settings.endpoint,
        "api_key": settings.api_key,
        "model": settings.model,
        "api_type": "azure",
        "api_version": settings.api_version
    }
]

llm_config = {
    "model": settings.model,
    "temperature": 0,
    "config_list": config_list,    
    #"cache_seed": None, # Enable caching
}

agent = ConversableAgent(
    name="chatbot",
    llm_config=llm_config,
    code_execution_config=False,  # Turn off code execution, by default it is off.
    function_map=None,  # No registered functions, by default it is None.
    human_input_mode="NEVER",  # Never ask for human input.
)

## Generate a python script to find the 1001st prime number

In [ ]:
reply = agent.generate_reply(messages=[{"role":"user","content":"Write a python script to find the 1001st prime number."}])
print(reply)

## Extract the Python code, execute it, and print the result

In [ ]:
# Using regex get the python code in the markdown
import re
code_groups = re.search(r'```python\n(.*?)```', reply, re.DOTALL)

if len(code_groups.groups()) > 0:
    code_execution_result = executor.execute_code_blocks(
            code_blocks=[
                CodeBlock(language="python", code=code_groups.group(1)),
            ]
        )

    print(code_execution_result)